<a href="https://colab.research.google.com/github/Gizmo-Labs/Statistical-Computing/blob/master/IdealerWohnort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Computing - Capstone Project

## Explorative Data Analysis